## OTE-28  MIRI image quality quick look analysis)

### User-defined parameters
The user should only have to edit this cell to point to the location of the inputs and miricoord pacakge on their machine and to select which filter to use.

In [1]:
# --------------------------------------------------------------------
# input/output parameters --------------------------------------------
# --------------------------------------------------------------------

# set the location of the simulation files **REQUIRED**


# set the filter for example analysis. Options are:
# F560W, F770W, F1000W, F1130W, F1280W, F1500W, F1800W, F2100W, F2550W
ima_filter = 'F560W'

# set filters to analyze and produce output files. Options are:
# F560W, F770W, F1000W, F1130W, F1280W, F1500W, F1800W, F2100W, F2550W
ima_filters_out = ['F560W', 'F770W', 'F1000W', 'F1130W', 'F1280W', 'F1500W', 'F1800W', 'F2100W', 'F2550W']

# output file directory (will be created if it doesn't exist)
output_dir = 'output/'

# --------------------------------------------------------------------
# global analysis parameters -----------------------------------------
# --------------------------------------------------------------------

# minimum flux density of catalog stars to be used (modify as needed). Currently set to a low value.
flux_min = 1.0e-10 

# parameter to set the absolute image value above which to select sources in the star finding algorithm. 
# The level is defined by sf_threshold*bkg_sigma
sf_threshhold = 3


### Import and setup packages

In [2]:
import os
import glob
import sys
#
import numpy as np
#
#from jwst import datamodels
#from jwst.datamodels import ImageModel
#
from matplotlib import style, pyplot as plt
from matplotlib import rcParams
from matplotlib.pyplot import figure
from matplotlib.backends.backend_pdf import PdfPages
#import chart_studio.plotly as py
from mpl_toolkits.mplot3d import Axes3D
#from bokeh.plotting import figure, show, output_file
#
from astropy.coordinates import match_coordinates_sky
from astropy.io import fits
from astropy.table import Table, Column
from astropy.coordinates import SkyCoord
from astropy.visualization import LogStretch, LinearStretch, PercentileInterval, ManualInterval
from astropy.io import ascii
import astropy.units as u
from astropy import wcs
from astropy.modeling import models
from astropy.stats import SigmaClip
#
import photutils
from photutils import psf, aperture_photometry
from photutils import Background2D, MedianBackground, CircularAperture, RectangularAperture
#
#from pystortion import crossmatch
#import pysiaf
#from miricoord.tel.tel_tools import jwst_radectov2v3
#from miricoord.tel.tel_tools import jwst_attmatrix

In [3]:
filter_fwhm = {"F560W", 2.00}
filter='F560W'

### Display input image

In [4]:
def display(image, title, **kwargs):

    xmax, ymax = image.shape
    xlim = kwargs.get('xlim', [0, xmax-1])
    ylim = kwargs.get('ylim', [0, ymax-1])

    img = image[xlim[0]:xlim[1]+1, ylim[0]:ylim[1]+1]

    fmin = kwargs.get('fmin', -0.01)   # Cut level as fraction of smax - smed
    fmax = kwargs.get('fmax', 1.0)   # Cut level as fraction of smax - smed

    smin = np.nanmin(img)
    smax = np.nanmax(img)
    smed = np.nanmedian(img)
    srng = smax - smed
    
    img = img / srng
    
    vmin = (smed/srng) + fmin
    vmax = (smed/srng) + fmax
    print("fmin, fmax = {:10.2f}{:10.2f}".format(fmin, fmax))
    print("smin, smax, smed, srng = {:10.3f}{:10.3f}{:10.3f}{:10.3f}".format(smin, smax, smed, srng))
    print("vmin, vmax = {:10.3f}{:10.3f}".format(vmin, vmax))

    stars = kwargs.get('stars', [])
    fig, ax = plt.subplots()
    xmin, xmax, ymin, ymax = xlim[0], xlim[1], ylim[0], ylim[1]
    fig = plt.imshow(img, 
                     extent=(xmin-0.5, xmax+0.5, ymin-0.5, ymax+0.5),
                     interpolation='nearest', cmap='binary',   # cmap='hot'
                     vmin=vmin, vmax=vmax, origin='lower')
    plt.colorbar()
    for star in stars:
        plt.xlim(xlim)
        plt.ylim(ylim)
        plt.scatter([star[1]], [star[2]], lw=0, s=4, color='r',marker='o')
    plt.title(str(filter) + ' - ' + title, fontsize='medium')
    return

In [5]:
def get_cv3_plot_parameters(code):
    """ Returns matplotlib colour and linestyle codes based on cv3 data id. """
    colours = {'F':'green', 'M':'blue', 'P':'red'}
    side = code[0]
    f_mm = float(code[1])
    colour = colours[side]
    lw = 2.0 if f_mm < 1.0 else 1.0
    ls = 'solid' if f_mm < 1.0 else 'dashed'
    return colour, lw, ls

### Create an image of the background

In [6]:
def make_bkg(img):
    sigma_clip = SigmaClip(sigma=3., iters=10)
    bkg_estimator = MedianBackground()
    bkg = Background2D(img, (50, 50), filter_size=(3, 3), sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)
    bkg_sigma=bkg.background_rms_median
    print('Background median = ', bkg.background_median)
    print('Background RMS = ',bkg_sigma)
    return bkg, bkg_sigma

### Find sources in the background subtracted image

In [7]:
def find_sources(img, bkg_sigma):
    filter_fwhm = 2.00
    dsf = photutils.DAOStarFinder(threshold=sf_threshhold*bkg_sigma, fwhm=filter_fwhm)
    found_stars = dsf(img)
    #found_stars = dsf(img1.data)
    found_stars['xcentroid'].name = 'x_0'
    found_stars['ycentroid'].name = 'y_0'
    found_stars['flux'].name = 'flux_0'
    print(len(found_stars), 'sources found')
    return found_stars

### Select bright stars in a sub-region of the image

In [8]:
def select_bright_stars(stars, **kwargs):
    # Clip bounds, xmin, xmax, ymin, ymax
    def_field = 550, 950, 50, 950
    threshold = kwargs.get('threshold', 70.0)
    field = kwargs.get('field', def_field)
    x1, x2, y1, y2 = field
    print('Selecting stars with flux > {:9.1f}'.format(threshold))
    print('  and in region x={:5.0f}-{:5.0f}, y={:5.0f}-{:5.0f}'.format(x1, x2, y1, y2))
    bstars = []
    for star in stars:
        x, y = star[1], star[2]
        noclip = (x1 < x < x2) and (y1 < y < y2)
        if (star[8] > threshold) and noclip:
            bstars.append(star)
    return bstars

### Calculate encircled energy fractions over a range of aperture radii for a list of stars

In [9]:
def find_eefs(radii, image, stars, eef_sim_list):

    n_pts = len(radii)
    for star in bstars:
        ident, x, y, flux = star[0], star[1], star[2], star[8]
        print('{:>5d}{:10.3f}{:10.3f}{:12.1f}'.format(ident, x, y, flux))
        centroid = x, y
        ee = np.zeros(n_pts)
        for i in range(0, len(radii)):
            r = radii[i]
            aperture = CircularAperture(centroid, r)
            phot = aperture_photometry(image, aperture)
            ee[i] = phot['aperture_sum']
        eef = ee / ee[n_pts-1]    # Normalise to last ee point
        eef_sim_list.append((radii, eef))
    return eef_sim_list

In [10]:
def plot_eefs(eef_list, code_list, f0rad, f0ee):
    import math 
    
    fig, ax = plt.subplots()
    plt.xlabel('log10(Radius / pixel)')
    plt.ylabel('Encircled energy fraction')
    
    xtick_lin_vals = np.array([0.1, 1, 2, 5, 10, 20])
    xtick_vals = np.log10(xtick_lin_vals)
    plt.xticks(xtick_vals, xtick_lin_vals)
    plt.xlim(np.log10([0.1, 20.0]))

    n_cv3 = len(code_list)
    i = 0
    for radii, eef in eef_list:
        
        rl = np.log10(radii)
        colour, lw, ls = 'grey', 1.0, 'solid'
        if i < n_cv3:
            colour, lw, ls = get_cv3_plot_parameters(code_list[i])
        if i > n_cv3:    # Kludge to suppress plotting out of focus data
            plt.plot(rl, eef, color=colour, ls=ls, lw=lw)
        i += 1

    radii, eef = eef_list[0]  # Plot CV3 in focus on top
    rl = np.log10(radii)
    colour, lw, ls = get_cv3_plot_parameters(code_list[0])
    plt.plot(rl, eef, color=colour, lw=lw, ls=ls)
    f0radl = math.log10(f0rad)
    plt.plot([f0radl], [f0ee], color=colour, marker='o')
    plt.text(math.log10(2.1), f0ee, '61.8 % @ r=1.9 pix.', color=colour, va='top', ha='left')

    return

### Exact rectangular aperture photometry

In [11]:
def exact_rectangular(image, aperture):
    cen = aperture.positions[0]
    w = aperture.w
    h = aperture.h
    x1 = cen[0] - w / 2.0
    x2 = x1 + w
    y1 = cen[1] - h / 2.0
    y2 = y1 + h
    c1, c2, r1, r2 = int(x1), int(x2), int(y1), int(y2)
    nr = r2 - r1 + 1       # Number of rows in subarray
    nc = c2 - c1 + 1
    wts = np.ones((nr, nc))
    im = image[r1:r1+nr, c1:c1+nc]

    fc1 = 1. - (x1 - c1)
    fc2 = x2 - c2
    if nc == 1:
        fc = fc1 + fc2 - 1.
        wts[:,0] *= fc
    else:
        wts[:,0] *= fc1
        wts[:,nc-1] *= fc2

    fr1 = 1. - (y1 - r1)
    fr2 = y2 - r2
    if nr == 1:
        fr = fr1 + fr2 - 1.
        wts[0,:] *= fr
    else:
        wts[0,:] *= fr1
        wts[nr-1,:] *= fr2

    wtim = np.multiply(im, wts)
    f = np.sum(wtim)
    return f

### Calculate integrated profile of a list of stars along row or column axis

In [12]:
def find_profiles(u_vals, v_coadd, image, stars, profile_list, axis):

    n_pts = len(u_vals)
    for star in bstars:
        ident, xcen, ycen, flux = star[0], star[1], star[2], star[8]
        print('{:>5d}{:10.3f}{:10.3f}{:12.1f}'.format(ident, xcen, ycen, flux))
        u_cen = xcen if axis == 'row' else ycen
        us = np.add(u_vals, u_cen)
        profile = np.zeros(n_pts)
        for i in range(0, n_pts):
            u = us[i]
            if axis == 'row':
                ap_pos = u, ycen
                aperture = RectangularAperture(ap_pos, w=u_sample, h=v_coadd)
            else:
                ap_pos = xcen, u
                aperture = RectangularAperture(ap_pos, w=v_coadd, h=u_sample)
            profile[i] = exact_rectangular(image, aperture)
        profile_list.append((u_vals, profile))
    return profile_list

In [13]:
def plot_profiles(profile_list, code_list, axis):
    fig, ax = plt.subplots()
    plt.xlabel('pixel')
    plt.ylabel('Profile')
    plt.title('Along ' + axis)
    plt.xlim([-5.0, 5.0])
    
    n_cv3 = len(code_list)
    i = 0
    for u_vals, profile in profile_list:
        x = u_vals - 0.5
        colour, lw, ls = 'grey', 1.0, 'solid'
        if i < n_cv3:
            colour, lw, ls = get_cv3_plot_parameters(code_list[i])
        y = profile / np.max(profile)
        plt.plot(x, y, color=colour, lw=lw, ls=ls)
        i += 1
    return

### Determine the input images for a given filter
This cell recursively searches the simulation directory for level 2B (cal) files then saves MIRI imager files for the user-defined filter to a list. 

Note that this cell will (annoyingly) take a some minutes to run depending on the number of files as it uses the datamodel metadata to check the detector and filter used. While this is not ideal, using the datamodels in the recursive search means that this code can be used for any set of files or filenaming conventions, e.g., MIRISim or data from the MAST archive.

In [16]:
#files = glob.iglob(os.path.join(simulation_dir, '**/*_cal.fits'), recursive=True)
#print(files)
#for f in files:
#    print(f)
%matplotlib notebook

rcParams['figure.figsize'] = [8., 5.]
plt.rcParams.update({'font.size': 12})

cv3_dir = 'data/cv3_data/'
cv3_prog = 'MIRM042MRF-'
cv3_list = [('F0', '0-6026132013_1_493_SE_2016-01-26T13h27m18', '0B-6026131649_1_493_SE_2016-01-26T13h23m38'),
            ('M2', 'M2-6026131344_1_493_SE_2016-01-26T13h21m48', 'M2B-6026131028_1_493_SE_2016-01-26T13h17m18'),
            ('M4', 'M4-6026130723_1_493_SE_2016-01-26T13h15m28', 'M4B-6026130412_1_493_SE_2016-01-26T13h10m58'),
            ('M6', 'M6-6026130058_1_493_SE_2016-01-26T13h09m08', 'M6B-6026125734_1_493_SE_2016-01-26T13h07m28'),
            ('M8', 'M8-6026125408_1_493_SE_2016-01-26T13h01m18', 'M8B-6026125042_1_493_SE_2016-01-26T12h58m08'),
            ('P2', 'P2-6026133128_1_493_SE_2016-01-26T13h37m48', 'P2B-6026132331_1_493_SE_2016-01-26T13h35m28'),
            ('P4', 'P4-6026133742_1_493_SE_2016-01-26T13h44m18', 'P4B-6026133431_1_493_SE_2016-01-26T13h41m38'),
            ('P6', 'P6-6026134415_1_493_SE_2016-01-26T13h50m58', 'P6B-6026134104_1_493_SE_2016-01-26T13h47m58'),
            ('P8', 'P8-6026135056_1_493_SE_2016-01-26T13h58m28', 'P8B-6026134735_1_493_SE_2016-01-26T13h58m38')]
cv3_post = '_LVL2.fits'
n_cv3 = len(cv3_list)
row_profile_list, col_profile_list, eef_list = [], [], []
code_list = []

# Define sampling arrays, can be different for CV3 and OTE-28.2 data sets
u_sample = 1.0  # Sample psf once per pixel to avoid steps
u_start = 0.0   # Offset from centroid
u_radius = 16.0 # Maximum radial size of aperture
u_vals = np.arange(u_start - u_radius, u_start + u_radius, u_sample)
v_coadd = 15.0  # Number of pixels to coadd orthogonal to profile

r_start, r_max, r_sample = 0.1, 16.0, 0.1
radii = np.arange(r_start, r_max, r_sample)



for obs in cv3_list:
    code = obs[0]
    code_list.append(code)
    cv3_img_file = cv3_dir + cv3_prog + obs[1] + cv3_post
    cv3_bgd_file = cv3_dir + cv3_prog + obs[2] + cv3_post

    hdu_list = fits.open(cv3_img_file)
    hdu = hdu_list[0]
    img, hdr = hdu.data[0], hdu.header

#    display(img, cv3_im1) 

    hdu_list = fits.open(cv3_bgd_file)
    hdu = hdu_list[0]
    bgd, hdr = hdu.data[0], hdu.header

    image = np.subtract(img, bgd)
    title = 'CV3 point source ' + code
    xlim = [350,650]
    ylim = [350,650]
    display(image, title, fmin=-0.01, fmax=0.03, xlim=xlim, ylim=ylim)

    # Find profiles for all bright stars and add to list
    bkg_sigma = 50.0
    stars = find_sources(image, bkg_sigma)
    for star in stars:
        print(star)
    field = xlim[0], xlim[1], ylim[0], ylim[1]
    bstars = select_bright_stars(stars, threshold=1.0, field=field)  # Filter bright stars >100 pix from edge of field

  
    row_profile_list = find_profiles(u_vals, v_coadd, image, stars, row_profile_list, 'row')
    col_profile_list = find_profiles(u_vals, v_coadd, image, stars, col_profile_list, 'col')
    eef_list = find_eefs(radii, image, stars, eef_list)
        
    

fmin, fmax =      -0.01      0.03
smin, smax, smed, srng =     -7.677   992.654     0.005   992.649
vmin, vmax =     -0.010     0.030


<IPython.core.display.Javascript object>

1 sources found
 id        x_0               y_0             sharpness            roundness1            roundness2      npix sky        peak             flux_0              mag        
--- ----------------- ------------------ ------------------ --------------------- --------------------- ---- --- ----------------- ----------------- -------------------
  1 511.1402728335967 509.91245305814397 0.7042330355160026 -0.026183110421477925 -0.009019629630919202   25 0.0 992.6544189453125 6.136205040187633 -1.9697496576384432
Selecting stars with flux >       1.0
  and in region x=  350-  650, y=  350-  650
    1   511.140   509.912       992.7


IndexError: invalid index to scalar variable.

In [15]:
def find_eeradii(eef_list, code_list, radref):
    """ Find the encircled energy at a specific radius (pixels) using linear 
    interpolation 
    """ 
    eerefs = np.zeros(len(code_list))
#    print(radref)
#    print(eef_list)
    j = 0
    for radii, eef in eef_list:
        iz = np.where(radii > radref)
        i = iz[0][0]
        factor = (eef[i] - eef[i-1]) / (radii[i] - radii[i-1])
        eeref = eef[i-1] + (radref - radii[i-1]) * factor
        print(j, code_list[j], radref, factor, eeref)
        eerefs[j] = eeref
        j += 1
    return eerefs

In [16]:
%matplotlib notebook

simulation_dir = 'sim_data'
obs_pre = '/jw01464005001_01101_000'
obs_post = '_mirimage_cal.fits'
obs_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
rcParams['figure.figsize'] = [8., 5.]
plt.rcParams.update({'font.size': 11})

xlim = [500,800]
ylim = [500,800]

u_sample = 1.0  # Sample psf once per pixel to avoid steps
u_start = 0.0   # Offset from centroid
u_radius = 20.0 # Maximum radial size of aperture
u_vals = np.arange(u_start - u_radius, u_start + u_radius, u_sample)
v_coadd = 15.0  # Number of pixels to coadd orthogonal to profile

r_start, r_max, r_sample = 0.1, 16.0, 0.1
radii = np.arange(r_start, r_max, r_sample)

n_bright = 0
for obs in obs_list[0:1]:
    file = simulation_dir + obs_pre + obs + obs_post
    print(file)
    hdu_list = fits.open(file)
    hdu = hdu_list[1]
    img, hdr = hdu_list[1].data, hdu_list[1].header
    dq = hdu_list[2].data
    img[np.where(dq == 262660)]=np.nan
    img[np.where(dq == 262656)]=np.nan
    img[np.where(dq == 2359812)]=np.nan
    img[np.where(dq == 2359808)]=np.nan
#    display(img, file, fmin=-0.01, fmax=0.03, xlim=xlim, ylim=ylim) 
    bkg_obj, bkg_sigma = make_bkg(img)    # Make background image
    bkg = bkg_obj.background
#    display(bkg, 'Background')
    image = img - bkg
#    display(img_minbkg, 'Bgd subtracted')
    stars = find_sources(image, bkg_sigma)
    bstars = select_bright_stars(stars)  # Filter bright stars >100 pix from edge of field
    n_bright += len(bstars)
    display(image, "OTE-28.2 simulated", fmin=-0.01, fmax=0.03, stars=bstars, xlim=xlim, ylim=ylim)
    print(np.nanmin(image), np.nanmax(image))
            
    # Find profiles for all bright stars and add to list
    row_profile_list = find_profiles(u_vals, v_coadd, image, stars, row_profile_list, 'row')
    col_profile_list = find_profiles(u_vals, v_coadd, image, stars, col_profile_list, 'col')
    eef_list = find_eefs(radii, image, stars, eef_list)

print("No. of stars analysed = {:5d}".format(n_bright))
plot_profiles(row_profile_list, code_list, 'row')
plot_profiles(col_profile_list, code_list, 'column')
eers = find_eeradii(eef_list[0:1], code_list[0:1], 1.9)
plot_eefs(eef_list, code_list, 1.9, eers[0])
print('Done')

sim_data/jw01464005001_01101_00001_mirimage_cal.fits


TypeError: __init__() got an unexpected keyword argument 'iters'